# What is OLS? How to estimate model parameters?
Let $y \in \mathbb{R}^{n \times 1}$ the dependent variable, 
$X \in \mathbb{R}^{n \times m}$ the indendent variables (including the intercept), 
and $\beta \in \mathbb{R}^{m \times 1}$ the regression coefficients.

The residuals or errors $\epsilon \in \mathbb{R}^{n \times 1}$ are the difference between $y$ and the prediction $X\beta$ 

$$
\epsilon = y - X \beta
$$

and the **Mean Squared Error** is

$$
MSE(\beta) = \frac{1}{n} \epsilon^T \epsilon 
$$

The **gradient** of the MSE is

$$
\nabla \, MSE(\beta) = \frac{2}{n} (X^T X\beta - X^T y)
$$

The gradient would be $\nabla MSE(\hat{\beta}) = 0$ for the **optimum** solution $\hat{\beta}$

$$
0 = X^T X\hat{\beta} - X^T y \\
\Leftrightarrow \hat{\beta} = (X^T X)^{-1} - (X^T y)
$$

The Ordinary Least Square (OLS) method is basically a) setting the gradient of the MSE to zero, and b) solving after $\beta$.

# LU decomposition
Let $A=X^T X$, $x=\beta$ and $b=X^T y$.
At its core `numpy.linalg.solve` tries to solve $Ax=b$ 

$$
\min_x Ax - b
$$

wheras LU decomposition is applied to $A = LU = PA$ at some point in the program.

The python code is 

```
beta = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
```

The scipy is more explicit but slower because it calls LAPACK twice (`lu_solve` and `lu_factor`) and thus has more overhead.

```
beta = scipy.linalg.lu_solve(scipy.linalg.lu_factor(np.dot(X.T,X)), np.dot(X.T,y))
```

# Pseudo-Inverse
The `p` stands for **p**seudoinverse or Moore-**P**enrose inverse.
(Just use the mnemonic that fits your brain.)

The pseudoinverse is the inverse based on Singular Value Decomposition but with the correction for ill-conditioned matrices.

The task is to estimate

$$
\hat{\beta} = (X^T X)^{-1} - (X^T y)
$$

whereas the inverse $A^{-1} = (X^T X)^{-1}$ is required.

1. Conduct SVD: $A = U S V^T$
2. Compute the inverse: $A^{-1} = V S^{-1} U^T$

If $A A^T$ is an ill-conditioned or resp. singular matrix then some of $S$'s diagonal elements $diag(s_1, s_2, .., s_n)$ are close to zero and the division-by-zero problems arises. 
The pseudoinverse approach just sets $\frac{1}{s_i}=0$ if a element's value is below a certain threshold $s_i<tol$.

**When to apply pinv?**
* not at all because you are not supposed to conduct Linear Regression on ill-conditioned matrices (e.g. multicollinarity)
* there is nothing you can do about $X$ (e.g. your customer want exactly these variables included, it's the best data you got, ...)

# QR Decomposition
The regressions coefficients 

$$
\hat{\beta} = (X^T X)^{-1} - (X^T y)
$$

will be estimated by compute the inverse matrix $A^{-1} = (X^T X)^{-1}$ with QR Decomposition $A = Q R$

$$
A^{-1} = R^{-1} Q^T
$$

The advantage is that $R$ is a triangular matrix that can be easily inverted.

# Singular Value Decomposition (SVD)
The regressions coefficients 

$$
\hat{\beta} = (X^T X)^{-1} - (X^T y)
$$

will be estimated by compute the inverse matrix $(X^T X)^{-1}$ with Singular Value Decomposition (SVD).

SVD decompose a matrix $A$ into 

$$
A = U S V^T
$$

with $U U^T = I$ and $V V^T = I$ and $S$ a diagonal matrix.

**Numpy lstsq**
The third implementation applies Numpy's [lstsq](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.lstsq.html) is based on LAPACK's [_gelsd](http://www.netlib.org/lapack/explore-html/d7/d3b/group__double_g_esolve_ga94bd4a63a6dacf523e25ff617719f752.html) what applies SVD to solve problems of type

$$
\min_x || b - A x ||_2
$$

For a Linear Regression problem the variables are $A=X^T X$, $x=\beta$ and $b=X^T y$.

